In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm
from statistics import mean
import copy
import sys
!{sys.executable} -m pip install openpyxl

<h1>Generalized Antibody Array Process</h1>
<h2>Overall Process</h2>
<ul>
    <li>Collect and Parse Antibody Arrays to be Compared</li>
    <li>Calculate probability of being negative controls by subarray</li>
    <li>Normalize to negative controls by subarray</li>
    <li>Method 1. Normalize to lowest positive control value (average of Positive 3 values) by subarray</li>
    <li>Method 2. Normalize to each control based on the value of the point</li>
    <li>Calculate Log 2 Fold Change of interested differences</li>
    <li>Sample visualizations of what I'm doing/showing</li>
</ul>

<h2>Collect and Parse Antibody Arrays to be Compared</h2>
<h4>This is where we'll define most of our variables</h4>

In [48]:
#Add in your spreadsheet files, I'm using four different ones 

df_of = pd.read_excel("OF_lysate.xlsx")
df_yf = pd.read_excel("YF_lysate.xlsx")
df_om = pd.read_excel("OM_lysate.xlsx")
df_ym = pd.read_excel("YM_lysate.xlsx")

all_dfs = [df_of, df_yf, df_ym, df_om] #This is super important! List all your arrays that you will want to compare here after import
amt_dfs = len(all_dfs)

#Reading one of the files, make sure it imported properly! You'll most likely have to take out the metadata for it to work
df_yf.head()

,Block,Column,Row,Name,ID,X,Y,Dia.,F532 Median,F532 Mean,...,Sum of Medians (1/532),Sum of Means (1/532),Log Ratio (1/532),F532 Median - B532,F532 Mean - B532,F532 Total Intensity,SNR 532,Flags,Normalize,Autoflag
0,1,1,1,Positive 1,Positive 1,4840,55650,180,21089,15930,...,21056,15897,Error,21056,15897,4078062,1589.500,0,0,0
1,1,2,1,Positive 1,Positive 1,5250,55650,170,24824,19930,...,24789,19895,Error,24789,19895,4145424,196.941,0,0,0
2,1,3,1,Positive 2,Positive 2,5670,55630,190,6910,9423,...,6876,9389,Error,6876,9389,2412277,853.364,0,0,0
3,1,4,1,Positive 2,Positive 2,6080,55630,180,7111,10032,...,7077,9998,Error,7077,9998,2568078,999.600,0,0,0
4,1,5,1,Positive 3,Positive 3,6480,55630,170,2007,2269,...,1972,2234,Error,1972,2234,471870,223.300,0,0,0


In [26]:
#Create an array with just the protein ID and F532 Total Intensity (the only two columns we use in the analysis)
#Stored in a list of 2d Arrays

paired_data = [[] for i in range(amt_dfs)]
for dataframe_i in range(len(all_dfs)):
    for index, row in all_dfs[dataframe_i].iterrows():
        paired_data[dataframe_i].append([row['ID'], row['F532 Total Intensity']])

#Reading the list
#print(paired_data)

In [27]:
#Create a list of all negative control values for each array, human uses _NCTRL but your array might be different

negs = [[] for i in range(amt_dfs)]

for i in range(len(paired_data)):
    for row in paired_data[i]:
        if row[0] == "_NCTRL":
            negs[i].append(row[1])

#Reading the list (shouldn't be too long), check for outliers here 
#print(negs)

negs = [[val for val in dataframe if val != 243168] for dataframe in negs] #this is me removing an outlier

#Same list without outliers
#print(negs)

In [28]:
#Get the average of the negative controls (to normalize to later!) for each array 
neg_control = [stats.gmean(negs[i]) for i in range(len(negs))]

#Check that average makes sense, revise outliers list if not 
print(neg_control)

#remove the negative control values from the paired data, unnecessary to keep them in your final data! 
paired_data = [list(filter(lambda row: row[0] != "_NCTRL", paired_data[dataframe])) for dataframe in range(amt_dfs)]

#Check that paired data filters properly
#print(paired_data)

[2984.131556087873, 1366.491166294797, 1949.2115598024518, 4191.69166633983]


In [29]:
#Pair up antibody array duplicates into a single averaged value, we're going to be using mean but I added stdev to flag any outliers early and easily

single_data = [[] for i in range(amt_dfs)]

for i in range(len(paired_data)):
    for j in range(0, len(paired_data[i]), 2):
        pair = [paired_data[i][j][1], paired_data[i][j+1][1]]
        single_data[i].append([paired_data[i][j][0], mean(pair)])

#Check that paired values make sense! 
#single_data

<h2>Calculate probability of being negative controls by subarray</h2>
<h4>This checks whether the results with the most amount of relative change are both positive, and gates if a result stops being made at all in different conditions</h4>

<p>The way I do it in this analysis is bootstrap the negative controls to create a probability space where I can calculate the probability something is a false positive, and the p value for that. In volcano graphs this is shown as -log10 p value. We'll normalize a step after so if you feel this is unnecessary skip the code block, it doesn't change anything in later steps</p>

In [30]:
negs_series = [pd.Series(neg) for neg in negs]
neg_spread = [subarray.sample(5000, replace=True) for subarray in negs_series]
results = [[] for i in range(amt_dfs)]

for i in range(amt_dfs):
    mu, std = norm.fit(negs_series[i])
    
    id = [datapoint[0] for datapoint in single_data[i]]
    observed_value = [datapoint[1] for datapoint in single_data[i]]
    
    results[i] = pd.DataFrame({
        'id': id,
        'observed_value': observed_value,
        'prob_negative': norm.cdf(observed_value, loc=mu, scale=std)
    })
    
    results[i].sort_values("prob_negative")
    
    results[i]["p_value"] = 1 - results[i]["prob_negative"]
    results[i]["-log10 p_value"] = -np.log10(results[i]["p_value"] + 1e-10)
    
    results[i] = results[i].drop(columns="prob_negative")
    results[i].sort_values("id", ascending=True)

#print each dataframe if you would like
# print(results[0].head())

#make one df with everything that can be exported 
merged_df = pd.concat(results, axis=1, ignore_index=True)

#
#IMPORTANTTTTT!!!!!!!! For anyone using this for their arrays, it's not important for understanding the code
#

#rename columns after merge in order of how you loaded the arrays in the second code block, I used my df names of, yf, ym, om
rename_list = ['OF id', 'OF observed_value', 'OF prob_negative', 'OF -log10 p_value', 'YF id', 'YF observed_value', 'YF prob_negative', 'YF -log10 p_value', 'YM id', 'YM observed_value', 'YM prob_negative', 'YM -log10 p_value', 'OM id', 'OM observed_value', 'OM prob_negative', 'OM -log10 p_value']
merged_df.columns = rename_list

#
#IMPORTANTTTTT!!!!!!!! For anyone using this for their arrays, it's not important for understanding the code
#

#double check, does this make sense?
#print(rename_list)
#print(merged_df)

#export df to a csv file so you can use it in graphs in your favorite graph generator
results_df = pd.DataFrame(data = merged_df)
results_df = results_df[(results_df["OF id"] != "Positive 1") & (results_df["OF id"] != "Positive 2") & (results_df["OF id"] != "Positive 3") & (results_df["OF id"] != "_NCTRL")]
results_df = results_df[["OF id", "OF -log10 p_value", "OM -log10 p_value", "YF -log10 p_value", "YM -log10 p_value"]].rename(columns={"OF id": "Proteins"})
print(results_df.head(30))

#this is for my analysis only 
results_df["Average Young"] = (results_df["YF -log10 p_value"] + results_df["YM -log10 p_value"])/2
results_df["Average Old"] = (results_df["OF -log10 p_value"] + results_df["OM -log10 p_value"])/2
results_df["Max value"] = results_df[["Average Young", "Average Old"]].max(axis=1)

results_df = results_df[["Proteins", "Average Young", "Average Old", "Max value"]]

results_df.to_csv("false_positive_probability.csv")
results_df.head(30)

                     Proteins  OF -log10 p_value  OM -log10 p_value  \
3                     6Ckine†           4.015890           9.999989   
4                  Activin A†           3.653486           8.480965   
5                   Activin B           4.241150           5.430277   
6                   Activin C          10.000000          10.000000   
7         Activin RIA / ALK-2          10.000000          10.000000   
8         Activin RIB / ALK-4           6.689898          10.000000   
9             Activin RII A/B           2.465328           3.381625   
10               Activin RIIA          10.000000          10.000000   
11       Adiponectin / Acrp30          10.000000          10.000000   
12                       AgRP          10.000000          10.000000   
13                      ALCAM           3.428555           5.921840   
14                 Angiogenin           1.106913           4.366235   
15             Angiopoietin-1           3.241112           4.336488   
16    

,Proteins,Average Young,Average Old,Max value
3,6Ckine†,10.000000,7.007939,10.000000
4,Activin A†,10.000000,6.067226,10.000000
5,Activin B,10.000000,4.835714,10.000000
6,Activin C,10.000000,10.000000,10.000000
7,Activin RIA / ALK-2,10.000000,10.000000,10.000000
8,Activin RIB / ALK-4,10.000000,8.344949,10.000000
9,Activin RII A/B,9.996714,2.923477,9.996714
10,Activin RIIA,10.000000,10.000000,10.000000
11,Adiponectin / Acrp30,10.000000,10.000000,10.000000
12,AgRP,10.000000,10.000000,10.000000


In [31]:
#print each dataframe if you would like
#print(results[0].head(40))
results[0].sort_values("observed_value", ascending=False)[results[0]["p_value"] < 0.0101]

/tmp/ipykernel_91/2440383777.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  results[0].sort_values("observed_value", ascending=False)[results[0]["p_value"] < 0.0101]


,id,observed_value,p_value,-log10 p_value
0,Positive 1,3541653.0,0.000000,10.000000
284,Positive 1,2895633.5,0.000000,10.000000
1,Positive 2,1811528.5,0.000000,10.000000
515,Positive 1,1553660.5,0.000000,10.000000
76,Coagulation Factor III / Tissue Factor,1514704.5,0.000000,10.000000
...,...,...,...,...
210,ICAM-1,10231.0,0.005538,2.256668
271,IL-13 R alpha 2,10220.5,0.005603,2.251576
112,ENA-78,10131.5,0.006185,2.208682
362,MMP-9†,9742.5,0.009401,2.026807


<h2>Normalize to negative controls by subarray</h2>
<h4>Here this just means divide subarray by average of negative controls, can also subtract, not sure which makes more sense here so I coded options </h4>

In [32]:
#WARNING: Rerun all cells rather than running this cell multiple times, it'll keep dividing smaller and smaller since we're not updating a copy

#To divide 
# for i in range(len(single_data)):
#     for row in single_data[i]:
#         row[1] = row[1] / neg_control[i]

# for i in range(len(paired_data)):
#     for row in paired_data[i]:
#         row[1] = row[1] / neg_control[i]    

#To substract 
for i in range(len(single_data)):
    for row in single_data[i]:
        row[1] = row[1] - neg_control[i]

for i in range(len(paired_data)):
    for row in paired_data[i]:
        row[1] = row[1] / neg_control[i] 

#Check it seems properly divided/subtracted 
#print(single_data)

<h2>Normalize to lowest positive control value (average of Positive 3 values) by subarray</h2>
<h4>This will involve finding the average of the lowest positive control values, then dividing values by it.
The idea of this is to get a "unit" of measurement that's the lowest positive unit so differences make sense</h4>

In [33]:
#Creating copies of anything that might get overridden because there are two normalization options

paired_data_1 = copy.deepcopy(paired_data)
single_data_1 = copy.deepcopy(single_data)

In [34]:
#Create a list of all the lowest positive control values for each array, human uses Positive 3 but your array might be different

poss = [[] for i in range(amt_dfs)]

for i in range(amt_dfs):
    for row in paired_data_1[i]:
        if row[0] == "Positive 3":
            poss[i].append(row[1])

#Reading the list (shouldn't be too long), check for outliers here 
print(poss)

#poss = [[val for val in dataframe if val != 243168] for dataframe in poss] #I have no outliers but if I did I'd do something like this

#Same list without outliers
#print(poss)

[[126.14189184523859, 142.554372018937, 269.5035339039583, 297.02075238331076, 137.71209220371878, 142.7048345543718], [345.3150753103384, 385.0365175990405, 598.8951997538544, 673.5352724566708, 317.0536412428835, 316.023265024779], [202.5249634985791, 232.07742521588906, 367.8497577105833, 424.11045422067076, 194.6294634321011, 209.05734831641308], [136.60212763215645, 163.5690443325691, 282.6632048140591, 324.2564358715902, 147.9493362978007, 156.98077348675227]]


In [35]:
#Get the average of the positive 3 controls
pos_control = [stats.gmean(poss[i]) for i in range(len(poss))]

#Check that average makes sense, revise outliers list if not 
#Keep in mind that this was already normalized to the negative control so it might be a tad lower than expected
#print(pos_control)

#remove the positive 3 control values from the paired data, unnecessary to keep them in your final data since we're going to normalize to them
single_data_1 = [list(filter(lambda row: row[0] != "Positive 3", single_data_1[dataframe])) for dataframe in range(amt_dfs)]
paired_data_1 = [list(filter(lambda row: row[0] != "Positive 3", paired_data_1[dataframe])) for dataframe in range(amt_dfs)]

#Check that paired data filters properly
#print(single_data)
#print(paired_data)

In [36]:
#Normalize to Positive 3 by dividing positive 3 average 

#WARNING: Rerun all cells rather than running this cell multiple times, it'll keep dividing smaller and smaller since we're not updating a copy

for i in range(len(single_data_1)):
    for row in single_data_1[i]:
        row[1] = row[1] / pos_control[i]

for i in range(len(paired_data_1)):
    for row in paired_data_1[i]:
        row[1] = row[1] / pos_control[i]  

In [37]:
#Check that positive control 1 and 2 make sense to this normalization (should be a clear difference if you're within working protein content)

other_poss = [[] for i in range(amt_dfs)]

for i in range(amt_dfs):
    for row in single_data_1[i]:
        if row[0] == "Positive 1" or row[0] == "Positive 2":
            other_poss[i].append([row[0], row[1]])

#remove the other positive controls from data, unnecessary to keep them in your final data
single_data_1 = [list(filter(lambda row: row[0] != "Positive 1", single_data_1[dataframe])) for dataframe in range(amt_dfs)]
paired_data_1 = [list(filter(lambda row: row[0] != "Positive 1", paired_data_1[dataframe])) for dataframe in range(amt_dfs)]
single_data_1 = [list(filter(lambda row: row[0] != "Positive 2", single_data_1[dataframe])) for dataframe in range(amt_dfs)]
paired_data_1 = [list(filter(lambda row: row[0] != "Positive 2", paired_data_1[dataframe])) for dataframe in range(amt_dfs)]

#print(other_poss)
#print(single_data_1)

<h2>You can either normalize to Positive 3 or this other way</h2>
<h4>Take the average of positive 1, 2, and 3 as three y axis units, with negative control at 0, to make a linear expression</h4>
<p>Caveat is RayBiotech hasn't responded to me yet so I'm not sure where positive 1, 2, 3 are on the scale, but assuming they respond to me at some point the idea is if increasing fluorescence isn't linear then this will show it better because each positive is higher than the next by a (theoretically) known amount.</p>
<p>Basically, theoretically the values could be log change, right now I'm making each positive one unit so there's a linear difference between them which is slightly rougher than a smooth curve difference but I think the most accurate given how few control datapoints there are, see below math for a better explanation</p>
<p>For me, only one datapoint even got higher than positive control 3, so I'll probably stick with normalizing to the average of positive control 3 unless new information comes up about the positive control distributions</p>

In [38]:
#Resetting, we normalized to the negative controls but are resetting the normalization of the positive controls 
#Creating copies of the datasets so you can choose which normalization method makes the best sense in context
#Only using single data here because frankly that's all that's going to be output anyways

paired_data_2 = copy.deepcopy(paired_data)
single_data_2 = copy.deepcopy(single_data)

In [39]:
#Create a list of all positive controls by positive control, the human 507 one has 3 positive controls 
#For clarification, each list (ex. pos1) is a list of lists, a list of positive control 1 by each array so they're seperated 

pos1 = [[] for i in range(amt_dfs)]
pos2 = [[] for i in range(amt_dfs)]
pos3 = [[] for i in range(amt_dfs)]

for i in range(amt_dfs):
    for row in paired_data_2[i]:
        if row[0] == "Positive 1":
            pos1[i].append(row[1])
        elif row[0] == "Positive 2":
            pos2[i].append(row[1])
        elif row[0] == "Positive 3":
            pos3[i].append(row[1])

#Reading the list (shouldn't be too long), check for outliers here, see above to take out outliers
#print(pos1)
#print(pos2)
#print(pos3)

In [40]:
#Get the average of the positive controls that we'll normalize to
neg_control0 = 1
pos_control1 = [stats.gmean(pos1[i]) for i in range(len(pos1))]
pos_control2 = [stats.gmean(pos2[i]) for i in range(len(pos2))]
pos_control3 = [stats.gmean(pos3[i]) for i in range(len(pos3))]

#Check that average makes sense, revise outliers list if not 
#print(pos_control1)
#print(pos_control2)
#print(pos_control3)

#remove the positive control values from the paired data, unnecessary to keep them in your final data since we're going to normalize to them
single_data_2 = [list(filter(lambda row: row[0] != "Positive 1", single_data_2[dataframe])) for dataframe in range(amt_dfs)]
single_data_2 = [list(filter(lambda row: row[0] != "Positive 2", single_data_2[dataframe])) for dataframe in range(amt_dfs)]
single_data_2 = [list(filter(lambda row: row[0] != "Positive 3", single_data_2[dataframe])) for dataframe in range(amt_dfs)]

#Check that paired data filters properly
#print(single_data)

In [41]:
#this is where the math comes in and normalization begins! 

for dataframe in range(amt_dfs):
    for row in single_data_2[dataframe]: 
        if (row[1] < neg_control0):
            row[1] = 0
        elif (row[1] < pos_control3[dataframe]):
            row[1] = (row[1]-neg_control0)/(pos_control3[dataframe]-neg_control)
        elif (row[1] < pos_control2[dataframe]):
            row[1] = 1 + (row[1]-pos_control3[dataframe])/(pos_control2[dataframe]-pos_control3[dataframe])
        elif (row[1] < pos_control1[dataframe]):
            row[1] = 2 + (row[1]-pos_control2[dataframe])/(pos_control2[dataframe]-pos_control3[dataframe])
        else:
            row[1] = 3

#print(single_data_2)

<h2>Log Fold Change with our Normalization Method for Graphical and Data Analysis</h2>

In [42]:
#First turn our list of lists into dataframes

#Choose which normalization method you want to use (1 or 2)
single_data = single_data_1

df_of = pd.DataFrame(data = single_data[0])
df_yf = pd.DataFrame(data = single_data[1])
df_ym = pd.DataFrame(data = single_data[2])
df_om = pd.DataFrame(data = single_data[3])

df_of = df_of.sort_values(df_of.columns[0], ascending = True, key=lambda col: col.str.lower())
df_yf = df_yf.sort_values(df_yf.columns[0], ascending = True, key=lambda col: col.str.lower())
df_ym = df_ym.sort_values(df_ym.columns[0], ascending = True, key=lambda col: col.str.lower())
df_om = df_om.sort_values(df_om.columns[0], ascending = True, key=lambda col: col.str.lower())

#df_of

In [43]:
#Merge into one dataframe containing all of the dataframes (technically there's a prettier way to do this and the above codeblock but this also works, I can change it later!)

df_of = df_of.rename(columns={0: "Proteins", 1: "OF_mean"})
df_yf = df_yf.rename(columns={0: "Proteins", 1: "YF_mean"})
df_ym = df_ym.rename(columns={0: "Proteins", 1: "YM_mean"})
df_om = df_om.rename(columns={0: "Proteins", 1: "OM_mean"})

df = df_of.merge(df_yf, on="Proteins")
df = df.merge(df_ym, on="Proteins")
df = df.merge(df_om, on="Proteins")
df = df.sort_values(df.columns[0], ascending = True, key=lambda col: col.str.lower())

df.to_csv("output.csv") #Output final dataframe if you want to create graphs or do more computation or just show it off elsewhere 
df.head()

,Proteins,OF_mean,YF_mean,YM_mean,OM_mean
0,6Ckine†,58.931777,24.251110,85.553304,171.231024
1,Activin A†,55.766595,23.775622,82.580351,129.118457
2,Activin B,60.819428,18.067375,69.477694,101.513606
3,Activin C,215.191493,40.189519,111.615483,184.181059
4,Activin RIA / ALK-2,141.607466,32.100248,114.646464,198.772832


In [44]:
#Add averages, means of the arrays you want to compare, in this case I'm comparing old to young and male to and female

df["Average Old"] = (df["OM_mean"] + df["OF_mean"])/2 
df["Average Young"] = (df["YM_mean"] + df["YF_mean"])/2 
df["Average Female"] = (df["OF_mean"] + df["YF_mean"])/2 
df["Average Male"] = (df["YM_mean"] + df["OM_mean"])/2 

df["Average"] = df["Average Old"] + df["Average Young"] / 2

#df.head()

In [45]:
#Add log fold change calculation, I want to find log fold change of old compared to young so that's what I'm using! But you can add whatever comparisons you want

log_fold = df 
log_fold["L2FC"] = np.log2(log_fold["Average Old"]) - np.log2(log_fold["Average Young"] + 0.0000001)
log_fold

#For volcano plots 

#This will just make a dataframe with the protein and log fold change, nothing else for comparison, so you can copy paste to prism or a software that makes log fold change graphs 
log_fold_final = log_fold.loc[:, ["Proteins", "L2FC", "Average"]]
log_fold_final.to_csv("logfold.csv")

/srv/conda/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [46]:
#list in order of significance 

all_dfs_df = [pd.DataFrame(df) for df in all_dfs]
all_dfs_df2 = [df["F532 Total Intensity"] for df in all_dfs_df]
all_dfs_df2.append(all_dfs_df[0]["ID"])

significance_df = pd.concat(all_dfs_df2, axis=1, ignore_index=True)
significance_df.columns = ["OF", "YF", "YM", "OM", "Proteins"]

#normalized to positive 3 control

significance_df["OF"] = (significance_df["OF"]/neg_control[0]) / pos_control[0]
significance_df["YF"] = (significance_df["YF"]/neg_control[1]) / pos_control[1]
significance_df["YM"] = (significance_df["YM"]/neg_control[2]) / pos_control[2]
significance_df["OM"] = (significance_df["OM"]/neg_control[3]) / pos_control[3]

#calculate t test from significance_df, have to rework everything lol 

old = pd.DataFrame(list(significance_df["OF"]) + list(significance_df["OM"]))
young = pd.DataFrame(list(significance_df["YF"]) + list(significance_df["YM"]))
proteins = pd.DataFrame(list(significance_df["Proteins"]) + list(significance_df["Proteins"]))

sig = pd.concat([old, young, proteins], axis=1, ignore_index=True)
sig.columns = ["Old", "Young", "Proteins"]

sig_list = sig.groupby("Proteins").apply(lambda df: stats.ttest_ind(df['Old'], df['Young'], alternative='two-sided')[1]).sort_values()
sig_dataframe = pd.DataFrame(sig_list)

sig_dataframe.to_csv("significance.csv")
sig_dataframe.head()

/tmp/ipykernel_91/2428311220.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sig_list = sig.groupby("Proteins").apply(lambda df: stats.ttest_ind(df['Old'], df['Young'], alternative='two-sided')[1]).sort_values()


,0
Proteins,
IL-17C,1.724329e-07
HB-EGF,3.170862e-06
MMP-7†,2.830215e-05
MMP-8†,5.914439e-05
IL-1 ra†,7.682107e-05


<h2>End of Normalization and Calculations</h2>
<h4>I used Prism for visualization.</h4>